In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import sys
sys.path.append("/content/gdrive/MyDrive/Colab Notebooks")
#changing dir location to colab Notebooks


In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from scipy import stats

Reading csv and changing to data frame

In [4]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Ml_assignment/assignment2/BitcoinHeistData.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2916697 entries, 0 to 2916696
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   address    object 
 1   year       int64  
 2   day        int64  
 3   length     int64  
 4   weight     float64
 5   count      int64  
 6   looped     int64  
 7   neighbors  int64  
 8   income     float64
 9   label      object 
dtypes: float64(2), int64(6), object(2)
memory usage: 222.5+ MB


checking for null vlaue any 

In [ ]:
print(df.isnull().any().any())
print(df.isnull().sum().sum())
# no null vlaue found 

False
0


pre processing

In [5]:
lE = LabelEncoder()
df['label'] = lE.fit_transform(df['label'])
df['address'] = lE.fit_transform(df['address'])
df = df.sample(frac=1)
#changing label and address type from string to int

In [6]:
# Split the data into a training, validation,
# and testing set (70:15:15 ratio)
# total range 2916697 
# Now dividing 2916697 in 70:15:15 ratio

def split_data(df):
  temp = df.to_numpy()
  X = temp[:,:-1]
  Y = temp[:,- 1]
  Y = temp[:,- 1].reshape(Y.shape[0], 1)
  
  x_train = X[: 2041689, :]
  x_test = X[2041689: 2479193, :]
  x_val = X[2479193: , :]
  y_train = Y[: 2041689, :]
  y_test = Y[2041689: 2479193, :]
  y_val = Y[2479193: , :]

  return x_train, x_test, x_val,y_train,y_test,y_val



In [7]:
x_train ,x_test,x_val,y_train ,y_test,y_val = split_data(df)

Part A) 

In [8]:
def Decision_tree_Train(x_train,x_test,x_val,y_train,y_test,y_val,type):
  for i in [4, 8, 10, 15, 20]:
    tree = DecisionTreeClassifier(criterion=type, max_depth=i)
    y_pred = tree.fit(x_train, y_train).predict(x_test)
    accu = accuracy_score(y_test,y_pred, normalize = True)
    print(type," for depth-",i,"accuracy = ",accu)

In [ ]:
Decision_tree_Train(x_train ,x_test,x_val,y_train ,y_test,y_val,"gini")
#gini with depth 15 is giving us the best accuracy.
#0.9882903927735518

gini  for depth- 4 accuracy =  0.9858401294616735
gini  for depth- 8 accuracy =  0.9865532657987127
gini  for depth- 10 accuracy =  0.9876092561439438
gini  for depth- 15 accuracy =  0.9888092451726156
gini  for depth- 20 accuracy =  0.987645827238151


In [ ]:
Decision_tree_Train(x_train ,x_test,x_val,y_train ,y_test,y_val,"entropy")
#entrophy with depth 15 is giving us the best accuracy.
#0.988877815974254

entropy  for depth- 4 accuracy =  0.9857715586600351
entropy  for depth- 8 accuracy =  0.9861486980690463
entropy  for depth- 10 accuracy =  0.9874904000877707
entropy  for depth- 15 accuracy =  0.9889143870684611
entropy  for depth- 20 accuracy =  0.9875658279695728


Part-B

In [9]:
def train_train(x_train,y_train,x_test,y_test, typ, number=100):
  store_tree = []
  data = np.concatenate([x_train,y_train], axis=1)
  # making list of 100 DT according to need of question and stroing 
  #inside list of name store_tree
  for i in range(number):
    store_tree.append(DecisionTreeClassifier(criterion=typ, max_depth=3))
  
  for i in store_tree:
    # randomly select 50% data
    # limit have 50 percent mark of total size
    limit = int(data.shape[0] * 0.5) 
    np.random.shuffle(data)
    x = data[:,:-1]
    y = data[:,-1]

    x_limit = x[:limit,:]
    y_limit = y[:limit]

    i=i.fit(x_limit,y_limit)
  
  # now finding acuracy
  y_pred =[]
  for tree in store_tree:
    y_pred.append(tree.predict(x_test))
  # changing y_pred into nparray to use in accuracy model
  y_pred = np.array(y_pred)
  temp = np.array(stats.mode(y_pred))
  f_y_pred = temp[0,0,:]

  accu = accuracy_score(y_test,f_y_pred, normalize = True)
  print(typ," accuracy = ",accu)







In [ ]:
train_train(x_train,y_train,x_test,y_test, "gini", number=100)

gini  accuracy =  0.9858401294616735


In [ ]:
train_train(x_train,y_train,x_test,y_test, "entropy", number=100)

entropy  accuracy =  0.9855041325336454


Part-c

In [9]:
from sklearn.ensemble import AdaBoostClassifier

In [10]:
def ada_b(x_train,x_test,y_train,y_test,typ):
  accu = []
  y_test = y_test.reshape(y_test.shape[0])
  y_train = y_train.reshape(y_train.shape[0])
  for i in [4, 8, 10, 15, 20]:
    clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=15  , criterion=typ), n_estimators=i)
    clf.fit(x_train, y_train)

    y_pred = clf.predict(x_test)
    accu.append(accuracy_score(y_test, y_pred))
  return accu

In [11]:
temp = ada_b(x_train,x_test,y_train,y_test,"entropy");


In [12]:
print(temp)

[0.9886835320362786, 0.9863589818607372, 0.985901843183148, 0.9869921189291984, 0.9880321094207138]
